In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as nnf
from torchsummary import summary
import torchvision
import model.c2d as c2d
from torch.utils.data import Dataset
from torchnet.dataset.splitdataset import SplitDataset
import random
import os
import numpy as np
import glob

import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import math
import itertools
import sys
from torchvision.utils import save_image, make_grid
from torchvision.models import vgg19

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import math
import cv2
from datetime import datetime
import time
import pandas as pd



In [18]:
# Data Loader

vid_folder = 'G:/.shortcut-targets-by-id/1VaRYG8M-m7nfxKooARU6qTiHpVPhNHuV/out'
audio_file = 'G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/1649855468-1649880078.csv'
df = pd.read_csv(audio_file,header=None)
duration = 5
vid_files = []
vid_fps = 5
if not len(vid_files):
    for vid_file in os.listdir(vid_folder):
        vid_time_stamp = int(time.mktime(datetime.strptime(vid_file[0:-4], '%Y-%m-%d_%H-%M-%S').timetuple()))
        vid_obj = cv2.VideoCapture(os.path.join(vid_folder,vid_file))    
        vid_files.append([vid_obj,vid_obj.get(cv2.CAP_PROP_FRAME_COUNT),vid_time_stamp])

print('files read')
batch_size = 64
indices = np.random.randint(0,len(vid_files),size=batch_size)
x_batch = []
y_batch = []
for i in indices:
    vid_tuple = vid_files[i]
    last_second = int(vid_tuple[1]/vid_fps) - duration + 1
    if last_second < 0:
        continue

    # print(int(vid_tuple[1]/vid_fps))
    # print(last_second)
    vid_start_second =  np.random.randint(0,last_second+1)
    audio_start_tstamp = vid_tuple[2] + vid_start_second + math.ceil(duration/2)
    video_snippet_start_index = vid_start_second*vid_fps
    video_snippet_end_index = video_snippet_start_index + (duration*vid_fps)
    vid_tuple[0].set(cv2.CAP_PROP_POS_FRAMES,video_snippet_start_index)

    y = df[df[0] == audio_start_tstamp].drop(columns=[1]).to_numpy()
    if y.shape[0] < 1:
        continue

    frames = []
    for j in range(video_snippet_start_index,video_snippet_end_index):
        
        _,frame = vid_tuple[0].read()
        # print(frame.shape)
        frames.append(frame)
    x_batch.append(frames)
    y_batch.append(y[0])
    print(len(y_batch))

print(np.array(x_batch).shape)
print(np.array(y_batch).shape)





files read
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
(63, 25, 240, 480, 3)
(63, 32)


In [13]:
index = 74
vid_files[indices[index]]
img = Image.fromarray(x_batch[index][0])
img.show()
print(vid_files[indices[index]],y_batch[index])

[<VideoCapture 00000252271AE2B0>, 1499.0, 1649878188] 1649878463


1649876473

In [9]:
import pandas as pd
audio_file = 'G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/1649855468-1649880078.csv'
df = pd.read_csv(audio_file,header=None)

In [19]:
start_tstamp = 1649855472
end_tstamp = 1649855482
y = df[(df[0]>= start_tstamp) & (df[0] < end_tstamp)].drop(columns=[1]).to_numpy()
y.shape

(10, 32)

In [2]:

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(0.5, 0.5)])
batch_size  = 100 
dataset = torchvision.datasets.MNIST(root='\\data',download=True,transform=transform)
params = {'batch_size': batch_size,
          'shuffle': True}


loader = torch.utils.data.DataLoader(dataset,**params)

In [3]:
res = c2d.Resnet(1,10,[3,1,1])
res.model.train()
criterion_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(res.model.parameters(), lr = 2e-4)

while(1):
    for i, data in enumerate(loader):

        x = data[0]
        y = data[1]

        res.model.zero_grad()

        y_pred = res.model(x)
        loss = criterion_loss(y_pred,y)

        loss.backward()
        optimizer.step()

        if not i%10:
            print(loss.item())


2.2827272415161133
1.6136571168899536


KeyboardInterrupt: 

In [26]:
torch.zeros((32,78))

torch.float32

In [4]:
class ImageDataset(Dataset):
    def __init__(self, root):
        # Transforms for low resolution images and high resolution images
        self.transform = transforms.Compose(
            [
                transforms.Resize((240,240), Image.BICUBIC),
                transforms.ToTensor(),
            ]
        )

        self.files = sorted(glob.glob(root + "/*.png"))

    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)

        return {"x": img, "y": 1}

    def __len__(self):
        return len(self.files)


dset = ImageDataset('C:\\Users\\Sumeet\\Desktop\\Projects\\Vid2Sound\\data\\training')


In [5]:
partitions = {0:0.25,1:0.25,2:0.25,3:0.25}
split_dset = SplitDataset(dset,partitions)




train_loader = DataLoader(
    dset,
    batch_size=2,
    shuffle=True)


In [6]:
split_dset.select(3)



split_loader = DataLoader(
    split_dset,
    batch_size=2,
    shuffle=True)

# to_pil = transforms.ToPILImage()
# count = 0
# for data in split_loader:
#     for i in range(data['x'].shape[0]):
#         to_pil(data['x'][i]).show()
#     count += 1

# print(count)

# count = 0
# for data in train_loader:
#     count += 1

# print(count)

In [12]:

from model.c2d import Resnet
model = Resnet(3,1,[1,1,1])

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for i,data in enumerate(split_loader):
    optimizer.zero_grad()
    x = data['x']
    y = data['y'].type(torch.float16)
    outputs = model(x)[:,0]
    loss = loss_fn(outputs, y)
    loss.backward()
    optimizer.step()

torch.Size([2])
0.66796875
torch.Size([2])
0.658203125
torch.Size([2])
0.65478515625
torch.Size([2])
0.6455078125
torch.Size([1])
0.6337890625


In [8]:
import cv2
import numpy as np
import torch
vid_capture = cv2.VideoCapture('test/2022-04-13_12-24-35.mp4')
fps = 5
batch = []
frames = []
secs = 0
while(vid_capture.isOpened()):
    ret, frame = vid_capture.read()
    if ret == True:
        frames.append(frame)
        if len(frames) and not len(frames)%fps:
            secs += 1
            batch.append(np.concatenate(frame,axis=2))

            frames = []
    else:
        vid_capture.release()
        break

vid_array = np.stack(batch)


t = torch.Tensor(vid_array).type(torch.uint8)
print(t.element_size()*t.nelement())



TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [5]:
from datetime import datetime
import time
datetime_object = datetime.strptime('2022-04-13_12-24-35', '%Y-%m-%d_%H-%M-%S')
int(time.mktime(datetime_object.timetuple()))

1649867075

In [15]:
import os
from operator import itemgetter, attrgetter


def get_files_in_interval(strt,end,path):
    file_list = []
    for file in os.listdir(path):
        file_time = int(time.mktime(datetime.strptime(file[0:-4], '%Y-%m-%d_%H-%M-%S').timetuple()))
        if file_time >= strt and file_time < end:
            file_list.append([file_time,file,os.path.join(path,file)])
    return file_list


path1 ='G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/bosch'
path2 = 'G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/iteris'  
strt_time = 1649999413
end_time = 1650137713
fps = 5


file_list1 = get_files_in_interval(strt_time,end_time,path1)
file_list2 = get_files_in_interval(strt_time,end_time,path2)

print(len(file_list1))
print(len(file_list2))




2
7


In [16]:
file_list1

[[1650137222,
  '2022-04-16_15-27-02.mp4',
  'G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/bosch\\2022-04-16_15-27-02.mp4'],
 [1650137522,
  '2022-04-16_15-32-02.mp4',
  'G:/Shared drives/UF-AI-Catalyst/UF AI Code/test_data/bosch\\2022-04-16_15-32-02.mp4']]

In [14]:
# while(len(file_list2) and len(file_list1)):
#     if file_list1[0][0] > file_list2[0][0]:
#         ref_list = file_list1
#         scroll_list = file_list2
#     else:
#         ref_list = file_list2
#         scroll_list = file_list1

#     while(ref_list[0][0] >= scroll_list[0][0] and ref_list[0][0] < scroll_list[1][0]):

while(len(file_list2) and len(file_list1)):
    if file_list1[0][0] > file_list2[0][0]:
        efile = file_list2[0]
        lfile = file_list1[0]
    else:        
        efile = file_list1[0]
        lfile = file_list2[0]


    evid = cv2.VideoCapture(efile[2])
    fps = evid.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
    frame_count = int(evid.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps
    if (efile[0] + duration) > lfile[0]:
        lvid = cv2.VideoCapture(efile[2])
        evid.set(cv2.CV_CAP_PROP_POS_FRAMES,fps*(lvid[0]-evid[0]))
        




IndentationError: expected an indented block (Temp/ipykernel_15916/3693431966.py, line 2)